<a href="https://colab.research.google.com/github/KSG722/class2022Spring/blob/main/requests_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#!pip install gradio
#import gradio as gr
import requests
import pandas as pd

requests library 연결

In [2]:
requests.get('http://apis.data.go.kr/B552584/ArpltnStatsSvc/getCtprvnMesureLIst?itemCode=PM10&dataGubun=HOUR&pageNo=1&numOfRows=100&returnType=xml&serviceKey=서비스키')

<Response [200]>

공공데이터포털에서 한국환경공단_에어코리아_대기오염통계 서비스에서 요청메시지를 입력해 오픈 API를 가지고오는 함수

In [5]:
URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'
      'getCtprvnMesureSidoLIst?'
      'sidoName=서울'
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
response = requests.get(URL)

상동. 요청메시지를 URL이라는 변수로 두고 코드를 짤 때 가시성을 좋게 한 것. 

이처럼 오픈 api에서 정보를 가져오고 출력하는 방식을 rest api라고 부름.

첫번째 줄은 서비스 URL. 즉 서비스를 제공해주는 URL

두번째 줄은 세부적인 서비스 명령(상세기능). getCtprvnMesureLIst <- 시도별 실시간 평균정보 조회 (에어코리아_대기오염통계 서비스_기술문서_v1.0 참고). 참고로 마지막 '?'까지가 규약이므로 써줘야함.

세번째 줄부터 요청 메시지 명세. 중요한 요청 메시지는 searchCondition과 returnType. 매 hour마다의 데이터를 요청하고 json타입으로 출력. serviceKey는 비밀번호같은 거.

In [6]:
print(response)

<Response [200]>


reponse라는 변수를 출력. 200이란 값은 파이썬 문법 상 성공적으로 출력되었음을 의미.

#### http://jsonviewer.stack.hu/ (copy json here to view json)

In [ ]:
r = response.json()
r

returnTpye을 json으로 정했기에 출력 데이터값을 보고싶으면 위와 같이 json을 명시해주어야함.

위 함수 실행 시 서울 전역구의 대기오염 정도 데이터가 쭉 나열된다. 그러나 상당히 복잡하게 나열되어 있기에 보기가 힘듦. 따라서 결과값을 복사한 뒤에 http://jsonviewer.stack.hu/ 사이트에 입력해서 보면 편함.

사이트에 들어간 후 text부분에 복사한 것을 붙여넣고 viewer로 들어가면 JSON-response-body-item에 들어가면 0부터 24까지의 목록에 차례대로 데이터가 정렬됨

In [12]:
items = r['response']['body']['items']
#items
items[1]

{'cityName': '강동구',
 'cityNameEng': 'Gangdong-gu',
 'coValue': '0.2',
 'dataGubun': '',
 'dataTime': '2022-03-27 16:00',
 'districtCode': '02',
 'districtNumSeq': '002',
 'itemCode': '',
 'khaiValue': '',
 'no2Value': '0.004',
 'numOfRows': '10',
 'o3Value': '0.039',
 'pageNo': '1',
 'pm10Value': '30',
 'pm25Value': '10',
 'resultCode': '',
 'resultMsg': '',
 'returnType': 'json',
 'searchCondition': '',
 'serviceKey': '',
 'sidoName': '서울',
 'so2Value': '0.002',
 'totalCount': ''}

items란 변수를 r속의 response속의 body속에 items에 해당하는 데이터로 설정.

위에서 보았듯이 http://jsonviewer.stack.hu/ 사이트 내의 분류에 따라 변수가 설정되고 따라서 0 ~ 24번의 데이터가 각각 따로 설정된다. 즉 구에 따라서 list로 데이터가 설정. 따라서 items[1]을 하면 2번째 데이터인 강동구 데이터가 출력됨. 

참고로 각각의 데이터들은 list형식으로 연결되어 있지만 각각의 데이터 요소들은 dict 형식임을 알 수 있다. { }로 연결되어 있기 때문. 예를 들어  cityName이란 Key는 강동구라는 value로 연결되어 있음

이는 후술하겠지만 data frame형식의 데이터를 만들기 위한 선행작업

In [19]:
cityName = [item['cityName'] for item in items]
pm10Value = [item['pm10Value'] for item in items]
pm25Value = [item['pm25Value'] for item in items]
so2Value = [item['so2Value'] for item in items]
coValue = [item['coValue'] for item in items]
o3Value = [item['o3Value'] for item in items]
no2Value = [item['no2Value'] for item in items]

cityName = pd.Series(cityName)
pm10Value = pd.Series(pm10Value)   
pm25Value = pd.Series(pm25Value)
so2Value = pd.Series(so2Value)
coValue = pd.Series(coValue)
o3Value = pd.Series(o3Value)
no2Value = pd.Series(no2Value)

df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})

cityName이란 변수 :

1. for item in items : items에 있는 정보들을 item안에 넣는다. 총 25번 돌림.
2. item['cityName'] : item안에 반복되며 들어온 정보 중 Key가 cityName에 해당하는 정보들을 cityName이란 변수로 넣겠다.

그리고 그 밑에서 다시 cityName을 정의. cityName 정보들을 시리즈 함수로 넣어 column으로 정의한 변수.

마지막에 dataframe으로 정렬.

In [20]:
print(df)

   cityName pm10Value pm25Value so2Value coValue o3Value no2Value
0       강남구        24         8    0.003     0.3   0.048    0.006
1       강동구        30        10    0.002     0.2   0.039    0.004
2       강북구        24         8    0.002     0.2   0.047    0.003
3       강서구        21         7    0.003     0.3   0.048    0.006
4       관악구        19         6    0.003     0.3   0.048    0.008
5       광진구        29         9    0.004     0.2   0.049    0.006
6       구로구        37        12    0.003     0.3   0.045    0.006
7       금천구        22         6    0.003     0.3   0.050    0.008
8       노원구        30        13    0.002     0.3   0.047    0.004
9       도봉구        17         8    0.003     0.2   0.046    0.004
10     동대문구        26         9    0.003     0.2   0.051    0.006
11      동작구                  11    0.003     0.2   0.049    0.006
12      마포구        24         7    0.002     0.2   0.050    0.009
13     서대문구        17         4    0.002     0.3   0.051    0.005
14      서초

위에서 한 작업 print

In [22]:
def test(sidoName):
  URL = ('http://apis.data.go.kr/B552584/ArpltnStatsSvc/'
      'getCtprvnMesureSidoLIst?'
      'sidoName='+sidoName+
      '&searchCondition=hour'
      '&pageNo=1'
      '&numOfRows=100'
      '&returnType=json'
      '&serviceKey=miH%2BZXg85lQy4%2FkmhffvygXDIFiTwisriSLxtyECSzw6hxTVK7yI8QKIOc7mP7LEqCnhBGuj6iW1gZW65dum7A%3D%3D')
  
  response = requests.get(URL)
  r = response.json()
  items = r['response']['body']['items']

  cityName = [item['cityName'] for item in items]
  pm10Value = [item['pm10Value'] for item in items]
  pm25Value = [item['pm25Value'] for item in items]
  so2Value = [item['so2Value'] for item in items]
  coValue = [item['coValue'] for item in items]
  o3Value = [item['o3Value'] for item in items]
  no2Value = [item['no2Value'] for item in items]

  pd_cityName = pd.Series(cityName)
  pd_pm10Value = pd.Series(pm10Value)   
  pd_pm25Value = pd.Series(pm25Value)
  so2Value = pd.Series(so2Value)
  coValue = pd.Series(coValue)
  o3Value = pd.Series(o3Value)
  no2Value = pd.Series(no2Value)

  df = pd.DataFrame({ 'cityName': cityName, 'pm10Value': pm10Value, 'pm25Value': pm25Value, 'so2Value': so2Value, 'coValue': coValue, 'o3Value': o3Value, 'no2Value': no2Value})
  return df

In [25]:
df

,cityName,pm10Value,pm25Value,so2Value,coValue,o3Value,no2Value
0,강남구,24,8,0.003,0.3,0.048,0.006
1,강동구,30,10,0.002,0.2,0.039,0.004
2,강북구,24,8,0.002,0.2,0.047,0.003
3,강서구,21,7,0.003,0.3,0.048,0.006
4,관악구,19,6,0.003,0.3,0.048,0.008
5,광진구,29,9,0.004,0.2,0.049,0.006
6,구로구,37,12,0.003,0.3,0.045,0.006
7,금천구,22,6,0.003,0.3,0.050,0.008
8,노원구,30,13,0.002,0.3,0.047,0.004
9,도봉구,17,8,0.003,0.2,0.046,0.004


다시 실행

In [ ]:
!pip install gradio
import gradio as gr

gradio라는 번들은 pandas 등과 다르게 컴퓨터 내에 없음
따라서 install과정을 거친 후 ram으로 import해옴

In [24]:
iface = gr.Interface(test, gr.inputs.Dropdown(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '세종']), outputs=['dataframe'])
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://59962.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f668b5dbb10>,
 'http://127.0.0.1:7860/',
 'https://59962.gradio.app')

gradio로 데이터프레임 만들기